In [2]:
import lxml.etree as ET
from words import tei
import xmltodict as xd

In [3]:
data = tei.TeiReader('words/data/data.xml')

In [4]:
conceptschemes = ['word_type', 'pos', 'gender', 'subc', 'usg', 'morph_type']

In [5]:
def create_word(legacy_id, lang, orth, wordtype):
    lang_scheme,_ = SkosConceptScheme.objects.get_or_create(dc_title='language')
    lang_concept,_ = SkosConcept.objects.get_or_create(pref_label=lang)
    lang_concept.scheme.set([lang_scheme])
    word_type_scheme,_ = SkosConceptScheme.objects.get_or_create(dc_title='word type')
    word_type,_ = SkosConcept.objects.get_or_create(pref_label=wordtype)
    word_type.scheme.set([word_type_scheme])
    new_word,_ = Word.objects.get_or_create(
        legacy_id=legacy_id,
        word_type=word_type,
        orth=orth,
        language=lang_concept
    )
    return new_word    

In [6]:
def add_concept_as_fk(objectinstance, conceptschemes, valuelist, scheme='scheme', pref_label='pref_label'):
    """adds concepts to objects"""
    objects = []
    for x in conceptschemes:
        if valuelist.get(scheme) == x:
            label = valuelist.get(pref_label, 'no {} provided'.format(x))
            concept,_ = SkosConcept.objects.get_or_create(
                    pref_label=label
                )
            concept.scheme.add(
                    SkosConceptScheme.objects.get_or_create(dc_title=x)[0]
                )
            setattr(objectinstance, x, concept)
            objectinstance.save()
            objects.append(objectinstance)
    return objects

In [7]:
for x in conceptschemes:
    SkosConceptScheme.objects.get_or_create(
        dc_title=x
    )

In [8]:
entries = data.find_elements('entry')

In [15]:
for x in entries['hits'][0:25]:
    entry = (xd.parse(ET.tostring(x)))['entry']
    legacy_id = entry.get('@xml:id', 'NO ID')
    forms = entry['form']
    if isinstance(forms, list):
        for form in forms:
            if form.get('@type') == 'lemma':
                temp_word = create_word(
                    entry.get('@xml:id', 'NO ID'),
                    form['orth'].get('@xml:lang', 'no lang'),
                    form['orth'].get('#text', 'NO TEXT'),
                    'lemma'
                )
            elif form.get('@type') == 'variant':
                temp_variant = create_word(
                    entry.get('@xml:id', 'NO ID'),
                    form['orth'].get('@xml:lang', 'no lang'),
                    form['orth'].get('#text', 'NO TEXT'),
                    'variant'
                )
                temp_word.has_form.add(temp_variant)
        temp_word.save()
    else:
        form = forms
        temp_word = create_word(
                    entry.get('@xml:id', 'NO ID'),
                    form['orth'].get('@xml:lang', 'no lang'),
                    form['orth'].get('#text', 'NO TEXT'),
                    'lemma'
                )
        if form.get('form'):
            form = form.get('form')
            if isinstance(form, list):
                for form in form:
                    if form.get('@type') == 'lemma':
                        temp_word = create_word(
                            entry.get('@xml:id', 'NO ID'),
                            form['orth'].get('@xml:lang', 'no lang'),
                            form['orth'].get('#text', 'NO TEXT'),
                            'lemma'
                        )
            else:
                temp_variant = create_word(
                        entry.get('@xml:id', 'NO ID'),
                        form['orth'].get('@xml:lang', 'no lang'),
                        form['orth'].get('#text', 'NO TEXT'),
                        'variant'
                    )
                temp_word.has_form.add(temp_variant)
    senses = entry.get('sense', 'no sense')
    if isinstance(senses, list):
        for sense in senses:
            try:
                temp_sense = create_word(
                    entry.get('@xml:id', 'NO ID'),
                    cit.get('@xml:lang', 'NO TEXT'),
                    cit.get('quote', 'NO TEXT'),
                    cit.get('@type', 'no cit type')
                )
                temp_word.has_sense.add(temp_sense)
                temp_word.save()
            except:
                pass
        
    else:
        senses = senses['cit']
        for cit in senses:
            try:
                temp_sense = create_word(
                    entry.get('@xml:id', 'NO ID'),
                    cit.get('@xml:lang', 'NO TEXT'),
                    cit.get('quote', 'NO TEXT'),
                    cit.get('@type', 'no cit type')
                )
                temp_word.has_sense.add(temp_sense)
                temp_word.save()
            except:
                pass
                
    try:
        gramtypes = entry['gramGrp']
        if not isinstance(entry.get('gramGrp'), list):
            if isinstance(entry['gramGrp'].get('gram'), list):
                for gram in entry['gramGrp'].get('gram'):
                    if not isinstance(gram, str):
                        if gram.get('@type') == 'root':
                            temp_gram = create_word(
                                entry.get('@xml:id', 'NO ID'),
                                gram.get('@xml:lang', 'NO TEXT'),
                                gram.get('#text', 'NO TEXT'),
                                'root'
                            )
                            temp_word.has_gram.add(temp_gram)
                            temp_word.save()
                        else:  
                            new = add_concept_as_fk(temp_word, conceptschemes, gram, scheme='@type', pref_label='#text')
                        
    except KeyError:
        pass
    temp_word.save()